In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier
#from sklearn.linear_model import LinearRegression
import optuna

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# check on the panda version and its dependencies
# i run this from time to time to ensure all is up to date
pd.__version__
#pd.show_versions()

In [ ]:
# kfolds-5/train_folds_5.csv is an output from https://www.kaggle.com/code/andrewnuk/kfolds-5

df_train = pd.read_csv('/kaggle/input/kfolds-5/train_folds_5.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sampleSubmission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
# detect number of folds
fold_no = df_train['kfolds'].max() +1
fold_no

In [ ]:
# output files from https://www.kaggle.com/code/andrewnuk/models-level1-stacking/notebook

df1 = pd.read_csv("../input/models-level1-stacking/train_pred_1_FE_20220524.csv")
df1.columns = ["id", "pred_1"]
df2 = pd.read_csv("../input/models-level1-stacking/train_pred_2_FE_20220524.csv")
df2.columns = ["id", "pred_2"]
df3 = pd.read_csv("../input/models-level1-stacking/train_pred_3_FE_20220524.csv")
df3.columns = ["id", "pred_3"]

df_test1 = pd.read_csv("../input/models-level1-stacking/test_pred_1_FE_20220524.csv")
df_test1.columns = ["id", "pred_1"]
df_test2 = pd.read_csv("../input/models-level1-stacking/test_pred_2_FE_20220524.csv")
df_test2.columns = ["id", "pred_2"]
df_test3 = pd.read_csv("../input/models-level1-stacking/test_pred_3_FE_20220524.csv")
df_test3.columns = ["id", "pred_3"]

df_train = df_train.merge(df1, on="id", how="left")
df_train = df_train.merge(df2, on="id", how="left")
df_train = df_train.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

In [ ]:
df_train.head()

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

In [ ]:
def run(trial):
    fold = 0 # lets run over 1 fold only, not all the folds
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = df_train[df_train['kfolds'] != fold].reset_index(drop=True)
    xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])


# do i still need to set use_label_encoder='false'?

    model = XGBRegressor(
        random_state=0,
        tree_method="hist",
        eval_metric='auc',
        objective='binary:logistic',
        use_label_encoder='false',
        n_jobs=-1,
        n_estimators=10000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=1000, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    roc = roc_auc_score(yvalid, preds_valid)
    return roc

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(run, n_trials=10)

In [ ]:
study.best_params

In [ ]:
# 14/5/2022

# {'learning_rate': 0.04827433382927939,
#  'reg_lambda': 36.38854397255708,
#  'reg_alpha': 0.00020787799265006187,
#  'subsample': 0.7939584892843453,
#  'colsample_bytree': 0.7350355235544905,
#  'max_depth': 2}

# 0.972784862545458

In [ ]:
# 21/5/2022

# {'learning_rate': 0.03747017734195057,
#  'reg_lambda': 0.00039998088402975774,
#  'reg_alpha': 0.015311444035654846,
#  'subsample': 0.9313237060297096,
#  'colsample_bytree': 0.8135196342799582,
#  'max_depth': 2}

# 0.9900480410556488

In [ ]:
# 23/5/2022

# {'learning_rate': 0.03414055159418502,
#  'reg_lambda': 5.841507236625909e-07,
#  'reg_alpha': 1.2933505668277112e-05,
#  'subsample': 0.3180298158205705,
#  'colsample_bytree': 0.7009939864826797,
#  'max_depth': 7}

# 0.9900832566065393

In [ ]:
# 24/5/2022

# {'learning_rate': 0.027156139688948214,
#  'reg_lambda': 6.284693161900589e-05,
#  'reg_alpha': 6.763746178982408e-07,
#  'subsample': 0.7857662348327735,
#  'colsample_bytree': 0.9785863619292364,
#  'max_depth': 3}

# 0.9907230990193749

In [ ]:
# kfolds-5/train_folds_5.csv is an output from https://www.kaggle.com/code/andrewnuk/kfolds-5

df_train = pd.read_csv('/kaggle/input/kfolds-5/train_folds_5.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sampleSubmission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
# output files from https://www.kaggle.com/code/andrewnuk/models-level1-stacking/notebook

df1 = pd.read_csv("../input/models-level1-stacking/train_pred_1_FE_20220524.csv")
df1.columns = ["id", "pred_1"]
df2 = pd.read_csv("../input/models-level1-stacking/train_pred_2_FE_20220524.csv")
df2.columns = ["id", "pred_2"]
df3 = pd.read_csv("../input/models-level1-stacking/train_pred_3_FE_20220524.csv")
df3.columns = ["id", "pred_3"]

df_test1 = pd.read_csv("../input/models-level1-stacking/test_pred_1_FE_20220524.csv")
df_test1.columns = ["id", "pred_1"]
df_test2 = pd.read_csv("../input/models-level1-stacking/test_pred_2_FE_20220524.csv")
df_test2.columns = ["id", "pred_2"]
df_test3 = pd.read_csv("../input/models-level1-stacking/test_pred_3_FE_20220524.csv")
df_test3.columns = ["id", "pred_3"]

df_train = df_train.merge(df1, on="id", how="left")
df_train = df_train.merge(df2, on="id", how="left")
df_train = df_train.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

In [ ]:
df_train.head()

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

In [ ]:
def run_cb(trial):
    fold = 0 # lets run over 1 fold only, not all the folds
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 1e-2, 10)
    min_data_in_leaf = trial.suggest_int("min_data_in_leaf", 1, 250)
    depth = trial.suggest_int("depth", 1, 7)
    leaf_estimation_iterations=trial.suggest_int("leaf_estimation_iterations", 1 , 10) 
    subsample=trial.suggest_float("subsample", 1e-2, 1, log=True)

    xtrain = df_train[df_train['kfolds'] != fold].reset_index(drop=True)
    xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])


# do i still need to set use_label_encoder='false'?

    model = CatBoostClassifier(
        random_state=0,
        loss_function="CrossEntropy",
        eval_metric='AUC',
        bootstrap_type='Bernoulli',
        n_estimators=10000,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        min_data_in_leaf=min_data_in_leaf,
        depth=depth,
        leaf_estimation_iterations=leaf_estimation_iterations,
        subsample=subsample,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=1000, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict_proba(xvalid)[:, -1]
    roc = roc_auc_score(yvalid, preds_valid)
    return roc

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(run_cb, n_trials=10)

In [ ]:
study.best_params

In [ ]:
# 14/5/2022

# {'learning_rate': 0.01785878100577341,
#  'l2_leaf_reg': 4.04235486206517,
#  'min_data_in_leaf': 5,
#  'depth': 5,
#  'leaf_estimation_iterations': 1,
#  'subsample': 0.5162065044245909}

# 0.9727399956695909

In [ ]:
# 21/5/2022

# {'learning_rate': 0.030090670734864194,
#  'l2_leaf_reg': 9.007007254109197,
#  'min_data_in_leaf': 156,
#  'depth': 4,
#  'leaf_estimation_iterations': 10,
#  'subsample': 0.22381455661476138}

# 0.9899503993719659

In [ ]:
# 23/5/2022

# {'learning_rate': 0.038248260169775346,
#  'l2_leaf_reg': 3.3729308831930993,
#  'min_data_in_leaf': 114,
#  'depth': 3,
#  'leaf_estimation_iterations': 9,
#  'subsample': 0.6576236055970964}

# 0.9900696906307224

In [ ]:
# 24/5/2022

# {'learning_rate': 0.1357006347561937,
#  'l2_leaf_reg': 7.668752327125118,
#  'min_data_in_leaf': 142,
#  'depth': 4,
#  'leaf_estimation_iterations': 4,
#  'subsample': 0.7886238858489611}

# 0.9906441382320859

In [ ]:
# kfolds-5/train_folds_5.csv is an output from https://www.kaggle.com/code/andrewnuk/kfolds-5

df_train = pd.read_csv('/kaggle/input/kfolds-5/train_folds_5.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sampleSubmission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
# output files from https://www.kaggle.com/code/andrewnuk/models-level1-stacking/notebook

df1 = pd.read_csv("../input/models-level1-stacking/train_pred_1_FE_20220524.csv")
df1.columns = ["id", "pred_1"]
df2 = pd.read_csv("../input/models-level1-stacking/train_pred_2_FE_20220524.csv")
df2.columns = ["id", "pred_2"]
df3 = pd.read_csv("../input/models-level1-stacking/train_pred_3_FE_20220524.csv")
df3.columns = ["id", "pred_3"]

df_test1 = pd.read_csv("../input/models-level1-stacking/test_pred_1_FE_20220524.csv")
df_test1.columns = ["id", "pred_1"]
df_test2 = pd.read_csv("../input/models-level1-stacking/test_pred_2_FE_20220524.csv")
df_test2.columns = ["id", "pred_2"]
df_test3 = pd.read_csv("../input/models-level1-stacking/test_pred_3_FE_20220524.csv")
df_test3.columns = ["id", "pred_3"]

df_train = df_train.merge(df1, on="id", how="left")
df_train = df_train.merge(df2, on="id", how="left")
df_train = df_train.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

In [ ]:
df_train.head()

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

In [ ]:
def run_lgb(trial):
    fold = 0 # lets run over 1 fold only, not all the folds
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)    
    subsample = trial.suggest_float("subsample", 0.1, 1.0)    
    subsample_freq = trial.suggest_int("subsample_freq", 1, 10)    
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 350)
    min_child_samples = trial.suggest_int("min_child_samples", 1, 250)


    xtrain = df_train[df_train['kfolds'] != fold].reset_index(drop=True)
    xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])


# do i still need to set use_label_encoder='false'?

    model = lgb.LGBMClassifier(
        random_state=0,
        objective='binary',
        metric='auc',
        n_estimators=20000,
        learning_rate =learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        subsample_freq=subsample_freq,
        colsample_bytree=colsample_bytree,
        min_child_weight=min_child_weight,
        min_child_samples=min_child_samples,
)
    
    model.fit(xtrain, ytrain, early_stopping_rounds=1000, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict_proba(xvalid)[:, -1]
    roc = roc_auc_score(yvalid, preds_valid)
    return roc

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(run_lgb, n_trials=10)

In [ ]:
study.best_params

In [ ]:
# 14/5/2022

# {'learning_rate': 0.09487432843504863,
#  'reg_lambda': 1.8360888906724572e-08,
#  'reg_alpha': 0.0003764803574815724,
#  'subsample': 0.13870435688141589,
#  'subsample_freq': 8,
#  'colsample_bytree': 0.997890142817157,
#  'min_child_weight': 155,
#  'min_child_samples': 51}

# 0.9724389946784788

In [ ]:
# 21/5/2022

# {'learning_rate': 0.17818666911001538,
#  'reg_lambda': 0.01936480267082536,
#  'reg_alpha': 0.00040662863909439586,
#  'subsample': 0.8859563624407328,
#  'subsample_freq': 10,
#  'colsample_bytree': 0.695155159055407,
#  'min_child_weight': 149,
#  'min_child_samples': 48
 
# 0.9898674099724503

In [ ]:
# 23/5/2022

# {'learning_rate': 0.0885347675461033,
#  'reg_lambda': 1.2701699792245506e-05,
#  'reg_alpha': 6.357172866480366e-07,
#  'subsample': 0.4391508507045103,
#  'subsample_freq': 4,
#  'colsample_bytree': 0.952787779118162,
#  'min_child_weight': 251,
#  'min_child_samples': 196}

# 0.9899642526293053

In [ ]:
# 24/5/2022

# {'learning_rate': 0.11344474715935352,
#  'reg_lambda': 0.0009274217057765754,
#  'reg_alpha': 0.013111761237607475,
#  'subsample': 0.6006055511455205,
#  'subsample_freq': 1,
#  'colsample_bytree': 0.5587762899397076,
#  'min_child_weight': 41,
#  'min_child_samples': 167}

# 0.9904058041442192